# cleaning

In [1]:
import requests 
import json
import os
from getpass import getpass
import pandas as pd
from pandas import json_normalize
#import tweepy
import time
from datetime import datetime
import sys
sys.path.append('../')
import os
from src import functions as fn 
from riotwatcher import LolWatcher, ApiError
from datetime import time
from datetime import datetime


In [ ]:
df = pd.read_csv("../data/player_data.csv")

In [10]:
pd.set_option("display.max_columns", 108)
df.sample(4)

,gameDuration,gameMode,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
8,1609061,ARAM,27,0,1,27901,18,53,Blitzcrank,0,2,2848,2848,2848,38044,11,0,1,0,True,False,False,False,False,False,15601,13350,Invalid,0,1,0,6656,3157,3001,3111,4629,3801,2052,18,3,9,MIDDLE,0,2,2,241,37213,21216,15502,0,0,0,1,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",9452,4491,15922,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SOLO,0,57,44,46,39,33,32,7,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,100,NaN,71,1600,47836,26693,0,34554,3393,0,14,222,312,1,0,1170,985,3130,2,3,2,0,0,0,0,0,True
2,1125748,ARAM,14,0,0,18332,17,53,Blitzcrank,0,2,0,0,0,18471,12,0,1,0,True,False,False,False,False,True,13011,12100,Invalid,0,0,1,6655,3157,3020,3102,1058,1058,2052,16,3,10,NONE,0,2,2,134,30188,16192,9091,0,0,0,0,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",3296,1473,13122,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,21,23,18,5,4,26,32,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,100,NaN,31,1117,37015,18515,0,22780,1740,0,22,80,284,1,0,3530,850,566,0,0,4,0,0,0,0,0,False
10,1598082,CLASSIC,11,0,1,11127,13,236,Lucian,0,2,3112,17455,3112,3725,1,0,2,1,False,False,False,False,False,True,12557,9950,BOTTOM,2,2,0,6672,0,3006,6676,1053,1043,3340,16,1,8,BOTTOM,492,7,3,1180,4438,1780,3138,4,0,0,0,0,0,9,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",97885,10934,6150,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,CARRY,0,36,28,28,9,3,7,2,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,405,jalex141,False,200,BOTTOM,1,1589,115366,13606,0,9563,2535,555,169,15,36,3,1,13042,891,274,1,2,1,0,18,0,4,8,True
19,1515366,ARAM,18,0,3,27487,18,10,Kayle,0,2,5812,5812,5812,25728,8,0,1,0,False,False,False,False,False,False,18612,16000,Invalid,1,1,1,6673,3006,3091,3094,3153,3085,2052,21,3,9,MIDDLE,500,3,2,215,71163,12540,18362,0,0,0,1,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",108095,15747,13787,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,54,63,40,8,6,21,5,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,404,jalex141,False,200,NaN,3,1507,180804,28287,0,34034,7346,2249,162,52,206,5,0,1545,0,1883,4,4,2,0,0,0,0,0,True


In [2]:
dfr = pd.read_csv("../data/player_data_r.csv")
pd.set_option("display.max_columns", 108)

In [4]:
dfr.head(4)

,gameDuration,gameMode,gameStartTimestamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,1568191,CLASSIC,1633216425961,25,0,3,12258,14,111,Nautilus,0,2,2591,2900,2591,26790,3,0,1,0,True,False,False,False,False,False,10579,7500,UTILITY,0,2.0,0,3860,3068,3111,3075,0,0,3364,15,2,6,BOTTOM,15,3,2,942,24337,9300,5245,0,0,0,1.0,0,0,5,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",12220,4091,8102,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,27,29,21,6,7,14,3,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,100,UTILITY,60,1568,48278,14415,0,13900,1884,0,38,202,100,1,0,11720,1022,552,0,3.0,3,0,27,0,2,11,True
1,989819,CLASSIC,1633215030516,0,0,1,9042,12,875,Sett,0,1,1012,1012,1012,8593,0,0,0,0,False,False,False,False,False,True,4923,3200,TOP,0,0.0,0,1054,3211,3111,1036,0,0,3364,8,0,1,NONE,0,0,1,0,0,0,7358,0,0,0,0.0,0,0,6,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",37672,3201,3656,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,32,16,25,3,1,12,2,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,200,TOP,11,989,41723,4039,0,11627,133,0,97,121,0,1,0,4050,837,612,0,0.0,1,0,3,0,1,0,False
2,1609810,CLASSIC,1633213036614,3,0,0,10894,13,36,DrMundo,0,1,2648,3113,2648,29611,9,0,0,0,False,False,False,False,False,True,8812,8350,TOP,0,0.0,0,1054,3111,3068,3065,3801,0,3340,14,0,4,MIDDLE,0,0,1,366,49072,17432,14131,4,0,0,0.0,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5003, ...",42847,8299,18619,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,DUO,0,121,35,66,11,3,12,3,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,100,TOP,18,1609,91939,25732,0,33674,7364,0,121,201,283,1,0,19,0,924,0,0.0,3,0,8,0,0,6,False
3,2063639,CLASSIC,1633108458479,14,0,1,17007,17,236,Lucian,0,2,1908,8808,1908,11465,8,0,1,1,False,False,False,False,False,False,12295,11650,BOTTOM,0,0.0,3,3026,6673,6676,3006,3123,0,3340,20,1,9,BOTTOM,482,3,2,492,5455,1653,14801,12,0,1,0.0,0,0,9,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",103276,13972,9242,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,CARRY,0,57,38,30,12,4,7,5,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,200,BOTTOM,1,2063,116700,16019,0,24453,2563,413,134,30,266,2,0,7968,393,410,0,0.0,10,0,18,0,3,10,False


In [6]:
datetime.fromtimestamp(1109179)
strf(datetime.fromtimestamp(1631560758.474))

NameError: name 'strf' is not defined

In [7]:
timestamp = 1631560758
date_time = datetime.fromtimestamp(timestamp)

print("Date time object:", date_time)

d = date_time.strftime("%m/%d/%Y, %H:%M:%S")
print("Output 2:", d)


Date time object: 2021-09-13 21:19:18
Output 2: 09/13/2021, 21:19:18


In [14]:
d

'09/13/2021, 21:19:18'

In [8]:
def duration(mili):
    mili = int(mili/1000)
    sec = mili % 60; mili = mili//60
    minutes = mili % 60; mili = mili//60
    h = mili
    #time_ = (h,minutes,sec)
    date_time = time(h,minutes,sec)
    date_time = date_time.strftime("%H:%M:%S")
    return date_time

In [9]:
datetime.time(date_time)

datetime.time(21, 19, 18)

In [10]:
duration(1966152)

'00:32:46'

In [11]:
#["game_start"]
dfr.gameStartTimestamp = dfr.gameStartTimestamp.apply(lambda x: datetime.fromtimestamp(x/1000).strftime("%m/%d/%Y, %H:%M:%S"))
dfr.gameDuration = dfr.gameDuration.apply(duration)

dfr.head(5)

,gameDuration,gameMode,gameStartTimestamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,00:26:08,CLASSIC,"10/03/2021, 01:13:45",25,0,3,12258,14,111,Nautilus,0,2,2591,2900,2591,26790,3,0,1,0,True,False,False,False,False,False,10579,7500,UTILITY,0,2.0,0,3860,3068,3111,3075,0,0,3364,15,2,6,BOTTOM,15,3,2,942,24337,9300,5245,0,0,0,1.0,0,0,5,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",12220,4091,8102,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,27,29,21,6,7,14,3,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,100,UTILITY,60,1568,48278,14415,0,13900,1884,0,38,202,100,1,0,11720,1022,552,0,3.0,3,0,27,0,2,11,True
1,00:16:29,CLASSIC,"10/03/2021, 00:50:30",0,0,1,9042,12,875,Sett,0,1,1012,1012,1012,8593,0,0,0,0,False,False,False,False,False,True,4923,3200,TOP,0,0.0,0,1054,3211,3111,1036,0,0,3364,8,0,1,NONE,0,0,1,0,0,0,7358,0,0,0,0.0,0,0,6,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",37672,3201,3656,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,32,16,25,3,1,12,2,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,200,TOP,11,989,41723,4039,0,11627,133,0,97,121,0,1,0,4050,837,612,0,0.0,1,0,3,0,1,0,False
2,00:26:49,CLASSIC,"10/03/2021, 00:17:16",3,0,0,10894,13,36,DrMundo,0,1,2648,3113,2648,29611,9,0,0,0,False,False,False,False,False,True,8812,8350,TOP,0,0.0,0,1054,3111,3068,3065,3801,0,3340,14,0,4,MIDDLE,0,0,1,366,49072,17432,14131,4,0,0,0.0,0,0,1,0,"{'statPerks': {'defense': 5001, 'flex': 5003, ...",42847,8299,18619,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,DUO,0,121,35,66,11,3,12,3,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,100,TOP,18,1609,91939,25732,0,33674,7364,0,121,201,283,1,0,19,0,924,0,0.0,3,0,8,0,0,6,False
3,00:34:23,CLASSIC,"10/01/2021, 19:14:18",14,0,1,17007,17,236,Lucian,0,2,1908,8808,1908,11465,8,0,1,1,False,False,False,False,False,False,12295,11650,BOTTOM,0,0.0,3,3026,6673,6676,3006,3123,0,3340,20,1,9,BOTTOM,482,3,2,492,5455,1653,14801,12,0,1,0.0,0,0,9,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",103276,13972,9242,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,CARRY,0,57,38,30,12,4,7,5,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,200,BOTTOM,1,2063,116700,16019,0,24453,2563,413,134,30,266,2,0,7968,393,410,0,0.0,10,0,18,0,3,10,False
4,00:24:53,CLASSIC,"09/29/2021, 21:04:00",5,0,0,9893,12,3,Galio,0,2,1456,3954,1456,20259,3,0,0,0,Tru

In [21]:
dfr.groupby(['role','teamPosition','championName']).size()

role     teamPosition  championName
CARRY    BOTTOM        Ashe             1
                       Lucian           2
DUO      TOP           DrMundo          1
NONE     JUNGLE        Sejuani          1
         TOP           Poppy            1
         UTILITY       Maokai           1
SOLO     MIDDLE        Galio            1
         TOP           DrMundo          1
                       Sett             1
                       Yorick           1
SUPPORT  TOP           Sett             1
         UTILITY       Alistar          2
                       Bard             1
                       Lux              2
                       Maokai           8
                       Morgana          4
                       Nautilus        11
                       Poppy            1
                       Swain            7
                       Xerath           1
                       Ziggs            1
dtype: int64

In [22]:
dfr.role.value_counts().index[0]

'SUPPORT'

In [26]:
interesting_columns = ['gameDuration','gameStartTimestamp','assists','champExperience','damageDealtToBuildings','damageDealtToObjectives','damageDealtToTurrets','damageSelfMitigated','deaths','firstBloodAssist','firstBloodKill','goldEarned','inhibitorTakedowns','kills','summonerLevel','timeCCingOthers','timePlayed','totalDamageDealt','totalDamageDealtToChampions','totalMinionsKilled','totalTimeSpentDead','visionScore','win']

In [25]:
dfr_f = dfr[dfr.role == dfr.role.value_counts().index[0]]
dfr_f.head(5)

,gameDuration,gameMode,gameStartTimestamp,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,perks,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,00:26:08,CLASSIC,"10/03/2021, 01:13:45",25,0,3,12258,14,111,Nautilus,0,2,2591,2900,2591,26790,3,0,1,0,True,False,False,False,False,False,10579,7500,UTILITY,0,2.0,0,3860,3068,3111,3075,0,0,3364,15,2,6,BOTTOM,15,3,2,942,24337,9300,5245,0,0,0,1.0,0,0,5,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",12220,4091,8102,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,27,29,21,6,7,14,3,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,100,UTILITY,60,1568,48278,14415,0,13900,1884,0,38,202,100,1,0,11720,1022,552,0,3.0,3,0,27,0,2,11,True
1,00:16:29,CLASSIC,"10/03/2021, 00:50:30",0,0,1,9042,12,875,Sett,0,1,1012,1012,1012,8593,0,0,0,0,False,False,False,False,False,True,4923,3200,TOP,0,0.0,0,1054,3211,3111,1036,0,0,3364,8,0,1,NONE,0,0,1,0,0,0,7358,0,0,0,0.0,0,0,6,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",37672,3201,3656,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,32,16,25,3,1,12,2,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,406,jalex141,False,200,TOP,11,989,41723,4039,0,11627,133,0,97,121,0,1,0,4050,837,612,0,0.0,1,0,3,0,1,0,False
6,00:32:46,CLASSIC,"08/19/2021, 22:01:55",10,0,5,13029,15,111,Nautilus,0,2,1519,4228,1519,24684,4,0,1,0,False,False,False,False,False,False,10156,8100,UTILITY,0,1.0,0,3860,2031,3047,3068,3075,1033,3364,20,1,6,BOTTOM,0,5,2,416,28820,8422,7118,2,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5002, ...",15051,3755,6977,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,34,37,28,9,7,14,6,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,62,1958,50107,13051,0,14535,3462,0,48,274,83,1,0,6235,873,439,1,4.0,5,0,40,0,4,18,True
7,00:28:48,CLASSIC,"08/15/2021, 19:04:33",17,0,0,12454,14,25,Morgana,0,3,1540,5893,1540,7685,4,0,1,0,True,False,False,False,False,False,10018,9000,UTILITY,1,1.0,0,3853,2421,3020,3165,6653,3191,3364,19,1,6,BOTTOM,0,5,2,1151,53360,15683,7646,4,0,0,0.0,0,0,10,0,"{'statPerks': {'defense': 5001, 'flex': 5008, ...",5407,658,9083,3582,95xDwQpz_rN6dWpctehzQ-nc9j_w3oTvvkxF5FW56RCMMu...,0,NaN,NaN,SUPPORT,0,62,79,19,7,6,14,4,4,fuyafo-AAcCQWs6nJOJbV7ZCV2vtaziuVrMFM7ZC8DS6OP4,399,jalex141,False,200,UTILITY,50,1723,59212,16785,515,16730,4251,0,34,78,114,1,0,444,444,0,0,2.0,0,0,39,0,2,19,True
8,00:40:04,CLASSIC,"08/15/2021, 18:17:42",16,0,1,17218,17,50,Swain,0,3,3328,6371,3328,2

In [27]:
dfr_fr = dfr_f[interesting_columns]
dfr_fr

,gameDuration,gameStartTimestamp,assists,champExperience,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,firstBloodAssist,firstBloodKill,goldEarned,inhibitorTakedowns,kills,summonerLevel,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalMinionsKilled,totalTimeSpentDead,visionScore,win
0,00:26:08,"10/03/2021, 01:13:45",25,12258,2591,2900,2591,26790,3,True,False,10579,2.0,6,406,60,1568,48278,14415,38,100,27,True
1,00:16:29,"10/03/2021, 00:50:30",0,9042,1012,1012,1012,8593,0,False,False,4923,0.0,1,406,11,989,41723,4039,97,0,3,False
6,00:32:46,"08/19/2021, 22:01:55",10,13029,1519,4228,1519,24684,4,False,False,10156,1.0,6,399,62,1958,50107,13051,48,83,40,True
7,00:28:48,"08/15/2021, 19:04:33",17,12454,1540,5893,1540,7685,4,True,False,10018,1.0,6,399,50,1723,59212,16785,34,114,39,True
8,00:40:04,"08/15/2021, 18:17:42",16,17218,3328,6371,3328,26166,10,False,False,14976,1.0,10,399,83,2396,150400,62336,89,411,37,False
9,00:32:46,"08/15/2021, 17:39:25",18,13136,1405,5470,1405,39491,4,False,False,9657,1.0,2,398,72,1958,48077,11967,48,146,46,True
10,00:31:46,"08/15/2021, 17:01:41",13,10667,1783,1852,1783,12852,6,False,False,8728,0.0,0,398,57,1898,57017,19334,43,176,36,False
11,00:28:49,"08/15/2021, 16:27:19",14,11939,4896,7555,4896,24119,3,False,False,10180,0.0,3,398,60,1722,75920,15505,60,87,35,True
13,00:23:45,"04/13/2021, 20:42:04",6,7234,669,807,669,16831,4,False,False,6676,NaN,2,389,29,1425,30628,5461,40,69,22,False
15,00:42:34,"04/12/2021, 23:41:00",13,15348,463,4838,463,22820,14,False,False,11168,NaN,3,388,84,2554,70148,28477,44,467,36,False
